In [1]:
import pandas as pd
import numpy as np

In [2]:
from harmonic_inference.data.corpus_reading import aggregate_annotation_dfs
from pathlib import Path

ANNOTATIONS_PATH = Path('../corpora/annotations')
OUT_DIR = Path('corpus_data')

aggregate_annotation_dfs(ANNOTATIONS_PATH, OUT_DIR)

100%|██████████| 749/749 [00:29<00:00, 25.55it/s]


In [2]:
from harmonic_inference.data.corpus_reading import read_dump
from pathlib import Path

files_df = read_dump(Path('corpus_data', 'files.tsv'), index_col=0)
measures_df = read_dump(Path('corpus_data', 'measures.tsv'))
chords_df = read_dump(Path('corpus_data', 'chords.tsv'))
notes_df = read_dump(Path('corpus_data', 'notes.tsv'))

In [3]:
from harmonic_inference.utils import corpus_utils as cu
import importlib
importlib.reload(cu)

# Remove measure repeats
if isinstance(measures_df.iloc[0].next, tuple):
    measures_df = cu.remove_repeats(measures_df)

# Add offsets
if not all([column in notes_df.columns for column in ['offset_beat', 'offset_mc']]):
    notes_df_offsets = cu.add_note_offsets(notes_df, measures_df)

# Merge ties
notes_df_merged = cu.merge_ties(notes_df_offsets, measures=measures_df)


In [4]:
notes_df_merged.loc[~notes_df_merged.tied.isna()]

mc   mn  onset duration gracenote nominal_duration scalar  \
file_id note_id                                                               
0       37         2    2    3/4      7/8       NaN              1/4    3/2   
        41         2    2      1      3/4       NaN              1/8      1   
        43         2    2    9/8      5/8       NaN              1/4    3/2   
        50         2    2   11/8      3/8       NaN              1/8      1   
        51         2    2  23/16     5/16       NaN             1/16      1   
...              ...  ...    ...      ...       ...              ...    ...   
748     1636     108  108    1/4     7/12       NaN              1/4    2/3   
        1638     108  108   5/12      1/6       NaN              1/8    2/3   
        1648     108  108    3/4      1/2       NaN              1/4      1   
        1662     109  109    3/4      1/3       NaN              1/4      1   
        1664     110  110      0      3/4       NaN              1/2      1   

                 tied  tpc  midi  staff  voice  volta  offset_mc offset_beat  
file_id note_id                                                               
0       37          1    3    81      1      1   <NA>          3         1/8  
        41          1    4    76      2      1   <NA>          3         1/4  
        43          1    7    73      2      2   <NA>          3         1/4  
        50          1    3    69      1      3   <NA>          3         1/4  
        51          1    4    64      1      2   <NA>          3         1/4  
...               ...  ...   ...    ...    ...    ...        ...         ...  
748     1636        1    9    63      1      1   <NA>        108         5/6  
        1638        1    1    55      2      1   <NA>        108        7/12  
        1648        1    3    57      2      1   <NA>        109         1/4  
        1662        1    1    55      2      1   <NA>        110        1/12  
        1664        1    2    50      2      2   <NA>        110         3/4  

[26046 rows x 15 columns]

In [5]:
notes_df_offsets.loc[748].loc[notes_df_offsets.loc[748].mc == 28]

,mc,mn,onset,duration,gracenote,nominal_duration,scalar,tied,tpc,midi,staff,voice,volta,offset_mc,offset_beat
note_id,,,,,,,,,,,,,,,
383,28,28,0,1/4,NaN,1/4,1,1,9,51,2,1,<NA>,28,1/4
384,28,28,0,1/4,NaN,1/4,1,1,3,69,1,2,<NA>,28,1/4
385,28,28,0,1/12,NaN,1/8,2/3,<NA>,7,73,1,1,<NA>,28,1/12
386,28,28,1/12,1/12,NaN,1/8,2/3,<NA>,6,78,1,1,<NA>,28,1/6
387,28,28,1/6,1/12,NaN,1/8,2/3,<NA>,7,73,1,1,<NA>,28,1/4
388,28,28,1/4,1/12,NaN,1/8,2/3,-1,9,51,2,1,<NA>,28,1/3
389,28,28,1/4,3/16,NaN,1/8,3/2,-1,3,69,1,2,<NA>,28,7/16
390,28,28,1/4,1/4,NaN,1/4,1,1,7,73,1,1,<NA>,28,1/2
391,28,28,1/3,1/12,NaN,1/8,2/3,<NA>,6,42,2,1,<NA>,28,5/12


In [9]:
measures_df

mc   mn  keysig timesig act_dur offset       repeats  \
file_id measure_id                                                          
0       0             1    1       3    12/8     3/2      0  firstMeasure   
        1             2    2       3    12/8     3/2      0           NaN   
        2             3    3       3    12/8     3/2      0           NaN   
        3             4    4       3    12/8     3/2      0           NaN   
        4             5    5       3    12/8     3/2      0           NaN   
...                 ...  ...     ...     ...     ...    ...           ...   
748     105         106  106       2     2/2       1      0           NaN   
        106         107  107       2     2/2       1      0           NaN   
        107         108  108       2     2/2       1      0           NaN   
        108         109  109       2     2/2       1      0           NaN   
        109         110  110       2     2/2     3/4      0     endRepeat   

                    volta barline  numbering_offset  dont_count  next  
file_id measure_id                                                     
0       0            <NA>     NaN              <NA>        <NA>     2  
        1            <NA>     NaN              <NA>        <NA>     3  
        2            <NA>     NaN              <NA>        <NA>     4  
        3            <NA>     NaN              <NA>        <NA>     5  
        4            <NA>     NaN              <NA>        <NA>     6  
...                   ...     ...               ...         ...   ...  
748     105          <NA>     NaN              <NA>        <NA>   107  
        106          <NA>     NaN              <NA>        <NA>   108  
        107          <NA>     NaN              <NA>        <NA>   109  
        108          <NA>     NaN              <NA>        <NA>   110  
        109          <NA>     NaN              <NA>        <NA>  None  

[71004 rows x 12 columns]

In [6]:
files_df

,corpus_name,file_name
0,Bach-Suites,BWV806_01_Prélude.tsv
1,Bach-Suites,BWV806_02_Allemande.tsv
2,Bach-Suites,BWV806_03_Courante_I.tsv
3,Bach-Suites,BWV806_04_Courante_II.tsv
4,Bach-Suites,BWV806_05_Double_I.tsv
...,...,...
744,WFBach-Sonatas,Bach_WF_F02_Sonate_C-Dur.tsv
745,WFBach-Sonatas,Bach_WF_F03_Sonate_D-Dur.tsv
746,WFBach-Sonatas,Bach_WF_F03_Sonate_D-Dur_01.tsv
747,WFBach-Sonatas,Bach_WF_F03_Sonate_D-Dur_02.tsv


In [15]:
chords_df.iloc[0]

mc                            1
mn                            1
onset                         0
label                      .A.I
alt_label                   NaN
globalkey                     A
localkey                      I
pedal                       NaN
chord                         I
numeral                       I
form                        NaN
figbass                     NaN
changes                     NaN
relativeroot                NaN
phraseend                   NaN
chord_type                    M
globalkey_is_minor        False
localkey_is_minor         False
chord_tones           (0, 4, 1)
added_tones                  ()
root                          0
bass_note                     0
volta                      <NA>
special                     NaN
Name: (0, 0), dtype: object

In [9]:
measures_df

mc   mn  keysig timesig act_dur offset       repeats  \
file_id measure_id                                                          
0       0             1    1       3    12/8     3/2      0  firstMeasure   
        1             2    2       3    12/8     3/2      0           NaN   
        2             3    3       3    12/8     3/2      0           NaN   
        3             4    4       3    12/8     3/2      0           NaN   
        4             5    5       3    12/8     3/2      0           NaN   
...                 ...  ...     ...     ...     ...    ...           ...   
748     105         106  106       2     2/2       1      0           NaN   
        106         107  107       2     2/2       1      0           NaN   
        107         108  108       2     2/2       1      0           NaN   
        108         109  109       2     2/2       1      0           NaN   
        109         110  110       2     2/2     3/4      0     endRepeat   

                    volta barline  numbering_offset  dont_count  next  
file_id measure_id                                                     
0       0            <NA>     NaN              <NA>        <NA>     2  
        1            <NA>     NaN              <NA>        <NA>     3  
        2            <NA>     NaN              <NA>        <NA>     4  
        3            <NA>     NaN              <NA>        <NA>     5  
        4            <NA>     NaN              <NA>        <NA>     6  
...                   ...     ...               ...         ...   ...  
748     105          <NA>     NaN              <NA>        <NA>   107  
        106          <NA>     NaN              <NA>        <NA>   108  
        107          <NA>     NaN              <NA>        <NA>   109  
        108          <NA>     NaN              <NA>        <NA>   110  
        109          <NA>     NaN              <NA>        <NA>  None  

[71004 rows x 12 columns]

In [10]:
notes_df

mc   mn onset duration gracenote nominal_duration scalar  \
file_id note_id                                                              
0       0          1    1     0     1/16       NaN             1/16      1   
        1          1    1  1/16     1/16       NaN             1/16      1   
        2          1    1   1/8     1/16       NaN             1/16      1   
        3          1    1  3/16     1/16       NaN             1/16      1   
        4          1    1   1/4     1/16       NaN             1/16      1   
...              ...  ...   ...      ...       ...              ...    ...   
748     1675     110  110  5/12     1/12       NaN              1/8    2/3   
        1676     110  110  5/12     1/12       NaN              1/8    2/3   
        1677     110  110   1/2      1/4       NaN              1/4      1   
        1678     110  110   1/2      1/4       NaN              1/4      1   
        1679     110  110   1/2      1/4       NaN              1/4      1   

                 tied  tpc  midi  staff  voice  volta  
file_id note_id                                        
0       0        <NA>    3    45      2      2   <NA>  
        1        <NA>    7    49      2      2   <NA>  
        2        <NA>    4    52      2      2   <NA>  
        3        <NA>    3    57      2      1   <NA>  
        4        <NA>    8    56      2      1   <NA>  
...               ...  ...   ...    ...    ...    ...  
748     1675     <NA>    3    57      2      1   <NA>  
        1676     <NA>    7    61      1      1   <NA>  
        1677       -1    2    50      2      2   <NA>  
        1678     <NA>    6    54      2      1   <NA>  
        1679     <NA>    2    62      1      1   <NA>  

[1004360 rows x 13 columns]

In [11]:
import harmonic_inference.data.piece as piece
import harmonic_inference.utils.harmonic_utils as hu

import importlib
importlib.reload(piece)
importlib.reload(hu)

score_piece = piece.ScorePiece(notes_df.loc[0], chords_df.loc[0], measures_df.loc[0])
for key in score_piece.get_keys():
    print(key)

A KeyMode.MAJOR
D KeyMode.MAJOR
A KeyMode.MAJOR
D KeyMode.MAJOR
E KeyMode.MAJOR
A KeyMode.MAJOR
E KeyMode.MAJOR
A KeyMode.MAJOR
E KeyMode.MAJOR
A KeyMode.MAJOR
E KeyMode.MAJOR
A KeyMode.MAJOR
E KeyMode.MAJOR
A KeyMode.MAJOR
E KeyMode.MAJOR
A KeyMode.MAJOR
E KeyMode.MAJOR
A KeyMode.MAJOR
E KeyMode.MAJOR
A KeyMode.MAJOR
E KeyMode.MAJOR
A KeyMode.MAJOR
E KeyMode.MAJOR
A KeyMode.MAJOR
D KeyMode.MAJOR
A KeyMode.MAJOR
F# KeyMode.MINOR
A KeyMode.MAJOR
F# KeyMode.MINOR
A KeyMode.MAJOR
E KeyMode.MAJOR
A KeyMode.MAJOR
F# KeyMode.MINOR
A KeyMode.MAJOR
F# KeyMode.MINOR
A KeyMode.MAJOR
F# KeyMode.MINOR
B KeyMode.MINOR
A KeyMode.MAJOR
B KeyMode.MINOR
A KeyMode.MAJOR
B KeyMode.MINOR
F# KeyMode.MINOR
B KeyMode.MINOR
A KeyMode.MAJOR
B KeyMode.MINOR
F# KeyMode.MINOR
B KeyMode.MINOR
A KeyMode.MAJOR
B KeyMode.MINOR
A KeyMode.MAJOR
B KeyMode.MINOR
A KeyMode.MAJOR
B KeyMode.MINOR
A KeyMode.MAJOR
B KeyMode.MINOR
A KeyMode.MAJOR
E KeyMode.MAJOR
B KeyMode.MINOR
A KeyMode.MAJOR
B KeyMode.MINOR
E KeyMode.MAJOR
A

In [9]:
chords_df.loc[94].loc[chords_df.loc[94].mc == 71]

,mc,mn,onset,chords,chord,key,pedal,numeral,form,figbass,...,relativeroot,phraseend,globalkey,bass_step,mc_next,onset_next,next_chord_id,chord_length,globalminor,localminor
chord_id,,,,,,,,,,,,,,,,,,,,,
14695,71,72,0,#viio6/III,#viio6/III,I,<NA>,#vii,o,6,...,III,<NA>,E,##4,71,1/4,14696,1/4,False,False
14696,71,72,1/4,III,III,I,<NA>,III,<NA>,<NA>,...,<NA>,<NA>,E,3,71,7/16,14697,3/16,False,False
14697,71,72,7/16,V7/V,V7/V,I,<NA>,V,<NA>,7,...,V,<NA>,E,2,72,1/16,14698,1/8,False,False


In [16]:
files_df.loc[102]

extension                               .mscx
filename                            08-1.mscx
md5          b69eef6286168c9d334cd29ce4bae9dd
name                                     08-1
rel_path        annotations/Beethoven-Sonatas
subcorpus                   Beethoven-Sonatas
Name: 102, dtype: object

In [ ]:
import harmonic_inference_data as hid
import importlib
importlib.reload(hid)

datasets = {'global': {},
            'local': {},
            'none': {}}

datasets['global']['train'], datasets['global']['valid'], datasets['global']['test'] = hid.get_train_valid_test_splits(
    chords_df=chords_df, notes_df=notes_df, measures_df=measures_df, files_df=files_df,
    seed=0, h5_directory='data', h5_prefix='globalkey_811split', make_dfs=True, transpose_global=True
)

datasets['local']['train'], datasets['local']['valid'], datasets['local']['test'] = hid.get_train_valid_test_splits(
    chords_df=chords_df, notes_df=notes_df, measures_df=measures_df, files_df=files_df,
    seed=0, h5_directory='data', h5_prefix='localkey_811split', make_dfs=True, transpose_local=True
)

datasets['none']['train'], datasets['none']['valid'], datasets['none']['test'] = hid.get_train_valid_test_splits(
    chords_df=chords_df, notes_df=notes_df, measures_df=measures_df, files_df=files_df,
    seed=0, h5_directory='data', h5_prefix='811split', make_dfs=True
)

In [ ]:
import harmonic_inference_models as him
import harmonic_utils
import importlib
importlib.reload(him)
from ablation import get_masks_and_names
import torch
import os

masks, mask_names = get_masks_and_names()
dir_name = 'results'

data = 'local'
index = -1
prefix = (data + '_') if data != 'none' else ''

mask = torch.tensor(masks[index])
mask_name = mask_names[index]
checkpoint_dir = os.path.join(dir_name, prefix + mask_name)
log_file_name = os.path.join(dir_name, prefix + mask_name + '.log')

model = him.MusicScoreModel(len(datasets[data]['test'][0]['notes'][0]), len(harmonic_utils.CHORD_TYPES) * 12,
                            dropout=0.2, input_mask=mask)

print(mask_name)
print(checkpoint_dir)

In [ ]:
import os
import model_trainer
import importlib
importlib.reload(model_trainer)

from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR

optimizer = Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.001)
criterion = CrossEntropyLoss()
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10)
schedule_var = 'valid_loss'

trainer = model_trainer.ModelTrainer(model, train_dataset=datasets[data]['train'], valid_dataset=datasets[data]['valid'],
                                     test_dataset=datasets[data]['test'], seed=0, num_epochs=100, early_stopping=20,
                                     optimizer=optimizer, scheduler=scheduler, schedule_var=schedule_var,
                                     criterion=criterion,
                                     log_every=1, log_file_name=log_file_name,
                                     save_every=10, save_dir=checkpoint_dir, save_prefix='checkpoint',
                                     resume=os.path.join(checkpoint_dir, 'best.pth.tar'))

In [ ]:
trainer.train()

In [ ]:
trainer = model_trainer.ModelTrainer(model, train_dataset=datasets[data]['train'], valid_dataset=datasets[data]['valid'],
                                     test_dataset=datasets[data]['test'], seed=0, num_epochs=100, early_stopping=20,
                                     optimizer=optimizer, scheduler=scheduler, schedule_var=schedule_var,
                                     criterion=criterion,
                                     log_every=1, log_file_name=log_file_name,
                                     save_every=10, save_dir=checkpoint_dir, save_prefix='checkpoint',
                                     resume=os.path.join(checkpoint_dir, 'best.pth.tar'))

loss, acc, outputs, labels = trainer.evaluate(valid=False)
print(loss, acc)

In [ ]:
import eval_utils as eu
import harmonic_utils as hu
import matplotlib.pyplot as plt

label_strings = hu.get_one_hot_labels()
conf_mat = eu.get_conf_mat(labels, outputs)

plt.figure(figsize=(30,30))
plt.imshow(conf_mat, interpolation='none')
plt.colorbar()
plt.xticks(ticks=list(range(len(label_strings))), labels=label_strings, rotation=90, fontsize=10)
plt.yticks(ticks=list(range(len(label_strings))), labels=label_strings, fontsize=10)
plt.show()

In [ ]:
import eval_utils as eu

correct, incorrect = eu.get_correct_and_incorrect_indexes(labels, outputs)
print('Correct: ' + str(len(correct)))
print('Incorrect: ' + str(len(incorrect)))

In [ ]:
import eval_utils as eu
    
eu.print_result(incorrect[0], labels, outputs, limit=10, prob=False)

In [ ]:
import eval_utils as eu

chord, onset_notes, all_notes = eu.get_input_df_rows(incorrect[0], datasets[data]['test'])

print(chord)
print("USED NOTES:")
print(onset_notes)
print()
print("ALL NOTES:")
print(all_notes)

In [ ]:
import matplotlib.pyplot as plt
import eval_utils as eu

correct_ranks, indexes_by_rank = eu.get_correct_ranks(labels, outputs)
    
plt.figure(figsize=(30,30))
plt.bar(range(len(outputs[0])), [len(indexes) for indexes in indexes_by_rank])

In [ ]:
import eval_utils as eu
import importlib
importlib.reload(eu)

eval_df = eu.get_eval_df(labels, outputs, datasets[data]['test'])
eval_df

In [ ]:
import ablation
import importlib
importlib.reload(ablation)

dfs = ablation.load_all_ablated_dfs(directory='results', prefix=prefix[:-1] if len(prefix) > 0 else None)
_, mask_names = ablation.get_masks_and_names()

In [ ]:
import pandas as pd
import os

logs = []
for mask_name in mask_names:
    logs.append(pd.read_csv(os.path.join(os.path.join('results', prefix + mask_name + '.log'))))

In [ ]:
for df, log, mask_name in zip(dfs, logs, mask_names):
    print(f"{mask_name} Acc: {100 * df.correct.sum() / len(df)}")
    print(log.iloc[-1])

In [ ]:
import eval_utils as eu

global_df = eu.load_eval_df('results/global_no_ablation.csv')
local_df = eu.load_eval_df('results/local_no_ablation.csv')
none_df = eu.load_eval_df('results/no_ablation.csv')

In [ ]:
global_df

In [ ]:
global_counts = global_df.groupby(['correct_chord'])['correct'].agg(['mean', 'count']).sort_values('count', ascending=False)
local_counts = local_df.groupby(['correct_chord'])['correct'].agg(['mean', 'count']).sort_values('count', ascending=False)
none_counts = none_df.groupby(['correct_chord'])['correct'].agg(['mean', 'count']).sort_values('count', ascending=False)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,5))
plt.scatter(global_counts['count'], global_counts['mean'], color='red', label='Global key')
plt.scatter(local_counts['count'], local_counts['mean'], color='blue', label='Local key')
plt.scatter(none_counts['count'], none_counts['mean'], color='yellow', label='No transposition')
plt.title('Global key transposed')
plt.xlabel('Count')
plt.ylabel('Accuracy')
plt.legend()
plt.show()